In [145]:
import os
from ast import literal_eval
import math
import json
import pandas as pd
import numpy as np
from pylab import *
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection

In [164]:
def calc_cost(distance):
    """
    determine the cost of the movement according to the rules:
    Each swing costs the particle $D^-2, where D is the straight-line distance of the length of rope used for that particular swing.
    """
    return pow(distance, -2)

def calc_distance(x1,y1,x2,y2): 
    """
    determine the distance between 2 points on a coordinate plane
    """
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def calc_slope(x1,y1,x2,y2):
    ##(0,0) => (5,0)
    if x1 == x2 and y1 == y2:
        return "NaL"
    elif x1 == x2:
        if y1 < y2:
            return "vertical_pos"
        elif y1 > y2:
            return "vertical_neg"
    elif y1 == y2:
        if x1 < x2:
            return "horizontal_pos"
        elif x1 > x2:
            return "horizontal_neg"
    else:
#         if slope > 0:
#             return str(slope)+"_pos"
#         elif slope < 0:
#             return str(slope)+"_neg"
        slope = (y2-y1)/(x2-x1)
    
    return str(slope)+"_"+quandrant_loc(x1,y1,x2,y2)
    
def plot_path(x1,y1,x2,y2, color="b"):
    # plot points
    ax.plot(x1, y1, color+'o')
    ax.plot(x2, y2, color+'o')
    # plot line
    ax.plot([x1, x2],[y1, y2], color+'-')

def lattice_intercepts(origin, radius):
    x = origin[0]
    y = origin[1]
    r_2 = round(pow(radius,2),1) # rounding to nearest 10th...
    xy_2 = (pow(x,2) + pow(y,2))
    lattice_intercepts = []
    i = 0
    while i <= xy_2:
        j = 0
        while j <= xy_2:
#             print(str(pow((i - x),2) + pow((j - y),2))+" == "+str(r_2))
            if (pow((i - x),2) + pow((j - y),2) == r_2):
                lattice_intercepts.append((i,j))
            j = j + 1
        i = i + 1
    return lattice_intercepts

def calc_angle(p0, p1, p2):
    """
    calculate the angle (in degrees) for vertex p0 p1 p2 
    """
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)
    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    return round(np.degrees(angle),4)

def inbounds(origin, thru_pt, distance, slope, angle=None):

    x = origin[0]
    y = origin[1]
    x_tp = thru_pt[0]
    y_tp = thru_pt[1]
    
    inbounds = True

    if slope == "vertical_pos":
        x1 = x
        y1 = y + distance
        if y1 > 20.5:
            inbounds = False
    elif slope == "vertical_neg":
        x1 = x
        y1 = y - distance
        if y1 < 0.5:
            inbounds = False
    elif slope == "horizontal_pos":
        x1 = x + distance
        y1 = y
        if x1 > 20.5:
            inbounds = False
    elif slope == "horizontal_neg":
        x1 = x - distance
        y1 = y
        if x1 < 0.5:
            inbounds = False
    else:
        slope = float(slope.split("_")[0])
        c = 1/math.sqrt(1+pow(slope,2))
        s = slope/math.sqrt(1+pow(slope,2))

        # QUADRANT 1
        if x < x_tp and y < y_tp:
            x1 = (x+distance*c)
            y1 = (y+distance*s)
        # QUADRANT 2
        elif x < x_tp and y > y_tp:
            x1 = (x+distance*c)
            y1 = (y+distance*s)
        # QUADRANT 3
        elif x > x_tp and y > y_tp:
            x1 = (x-distance*c)
            y1 = (y-distance*s)
        # QUADRANT 4
        elif x > x_tp and y < y_tp:
            x1 = (x-distance*c)
            y1 = (y-distance*s)

        if x1 < 0.5 or x1 > 20.5 or y1 < 0.5 or y1 > 20.5:
            inbounds = False
            
#     if inbounds:        
#         plot_path(x,y,x1,y1,"y")
#         plot_path(x,y,x_tp,y_tp,"c")
    
    return inbounds

def quandrant_loc(x,y,x1,y1):
    # QUADRANT 1
    if x < x1 and y < y1:
        return "q1"
    # QUADRANT 2
    elif x < x1 and y > y1:
        return "q2"
    # QUADRANT 3
    elif x > x1 and y > y1:
        return "q3"
    # QUADRANT 4
    elif x > x1 and y < y1:
        return "q4"

def recursive_swing(toss,start,v,graph,df,coords,og_post=None):
#     print(">>> START RECURSION!")
#     print(v)
#     print(len(coords))
#     print(df)
    g = 180
    d_rem = 0
    stack = (0,0,0,0,0)
    c = calc_cost(calc_distance(toss[0],toss[1],start[0],start[1]))
    
#     if (start[0], start[1]) in coords:
# #         print("removing things...")
#         coords[:] = [x for x in coords if x != (start[0], start[1])]

#     if (toss[0], toss[1]) in coords:
# #         print("removing things...")
#         coords[:] = [x for x in coords if x != (toss[0], toss[1])]
    
    
    for i in graph[(start[0],start[1])]:
        # make sure radius of swing long enough to catch a post and that it doesn't catch the origin
        if start[2] > i[2] and (start[0],start[1]) != (i[0],i[1]):
            g_2 = calc_angle((toss[0],toss[1]),(start[0],start[1]),(i[0],i[1]))
            if v == "pos":
                if g_2 < g: 
                    g = g_2
                    d_rem = start[2] - i[2]
                    stack = (i[0],i[1],d_rem,i[3],g_2)
            if v == "neg":
                g = g*-1
                if g_2 > g: 
                    g = g_2
                    d_rem = start[2] - i[2]
                    stack = (i[0],i[1],d_rem,i[3],g_2)
                    
    test_graph = [(i[0],i[1]) for i in graph[(start[0],start[1])]]

    # find all the lps
    if stack != (0,0,0,0,0) and inbounds((toss[0],toss[1]), (start[0],start[1]), start[2], calc_slope(toss[0],toss[1],start[0],start[1])):
        
        lps = lattice_intercepts((stack[0],stack[1]),stack[2]) 
        if len(lps) > 0:
            for l in lps:
                x = l[0]
                y = l[1]
                if (x,y) not in test_graph:
                    g_bound = calc_angle((toss[0],toss[1]),(start[0],start[1]),(stack[0],stack[1]))
                    g_swing = calc_angle((toss[0],toss[1]),(start[0],start[1]),(x,y))
                    if v == "pos":
                        if g_swing >= g_bound:
                            new_coords = list(coords)
#                             new_coords.remove((og_post[0],og_post[1]))
#                             new_coords[:] = [x for x in new_coords if x != (og_post[0],og_post[1])]
                            new_coords.append((x,y))
                            if og_post:
                                df.loc[len(df)] = [((og_post[0],og_post[1]),(x,y)),(excel_col(int(og_post[0]))+str(int(og_post[1])),excel_col(x)+str(y)), c, new_coords]
                    if v == "neg": 
                        if g_swing <= g_bound:
                            new_coords = list(coords)
#                             new_coords.remove((og_post[0],og_post[1]))
#                             new_coords[:] = [x for x in new_coords if x != (og_post[0],og_post[1])]
                            new_coords.append((x,y))
                            if og_post:
                                df.loc[len(df)] = [((og_post[0],og_post[1]),(x,y)),(excel_col(int(og_post[0]))+str(int(og_post[1])),excel_col(x)+str(y)), c, new_coords]
            # see if there is more slack...
            if stack[2] > 0:
                if og_post:
                    return recursive_swing(start,stack,v,graph,df,coords,og_post)
                else:
                    return recursive_swing(start,stack,v,graph,df,coords)
            else:
                return df
        else:
            return df
    else:
        return df
    
def excel_col(col):
    quot, rem = divmod(col-1,26)
    return excel_col(quot) + chr(rem+ord("A")) if col!=0 else ""


def build_graph(coords):
    graph = {}
    for i in range(len(coords)):
        x1 = coords[i][0]
        y1 = coords[i][1]
        viable_toss_stack = {}
        toss = []
        toss_length = 0
        for j in coords:
            x2 = j[0]
            y2 = j[1]
            distance = calc_distance(x1,y1,x2,y2)
            slope = calc_slope(x1,y1,x2,y2)
            if slope not in viable_toss_stack:
                viable_toss_stack[slope] = (x2,y2,distance,slope)
            else:
                if abs(viable_toss_stack[slope][2]) > abs(distance):
                    del viable_toss_stack[slope]
                    viable_toss_stack[slope] = (x2,y2,distance,slope)
                elif abs(viable_toss_stack[slope][2]) == abs(distance):
                    viable_toss_stack[slope] = (x2,y2,distance,slope)             


        for k in viable_toss_stack:
            toss.append((viable_toss_stack[k][0], viable_toss_stack[k][1], viable_toss_stack[k][2], viable_toss_stack[k][3]))
            graph[(x1,y1)] = toss
    return graph



# graph search loop
def find_moves(a,b,coords, move_df):
    
#     for i in gr:
#         x1 = i[0]
#         y1 = i[1]
    if (a, b) in coords:
        coords[:] = [x for x in coords if x != (a, b)]
        
    gr = build_graph(coords)
    ls
    
    for j in gr[(a,b)]:
        x2 = j[0]
        y2 = j[1]
        d2 = j[2]
        s2 = j[3]

        g_pos = 180
        d_pos_rem = 0
        g_pos_stack = (0,0,0,0,0)

        g_neg = -180
        d_neg_rem = 0
        g_neg_stack = (0,0,0,0,0)
#         if x1 == a and y1 == b:
        if (a,b) != (x2,y2) and s2 != "NaL":
            for k in gr[(x2,y2)]:
                x3 = k[0]
                y3 = k[1]
                d3 = k[2]
                s3 = k[3]
                g = calc_angle((a,b),(x2,y2),(x3,y3))

                if d3 < d2 and s3 != "NaL":

                    if 0 < g < g_pos:
                        g_pos = g
                        d_pos_rem = d2 - d3
                        g_pos_stack = (x3,y3,d_pos_rem,s3,g)
                    elif 0 > g > g_neg:
                        g_neg = g
                        d_neg_rem = d2 - d3
                        g_neg_stack = (x3,y3,d_neg_rem,s3,g)

            lps = lattice_intercepts((x2,y2), d2)

            if g_pos_stack != (0,0,0,0,0) and inbounds((x2,y2), (g_pos_stack[0],g_pos_stack[1]), d2, calc_slope(x2,y2,g_pos_stack[0],g_pos_stack[1])):
                g_p = g_pos_stack[4]
                for l in lps:
                    x4_p = l[0]
                    y4_p = l[1]
                    g2_p = calc_angle((a,b),(x2,y2),(x4_p,y4_p))
                    if 0 <= g2_p <= g_p and (x4_p,y4_p) != (a,b):
                        new_coords_pos = list(coords)
                        new_coords_pos[:] = [x for x in coords if x != (x2,y2)]
                        new_coords_pos.append((x4_p,y4_p))
                        move_df.loc[len(move_df)] = [((x2,y2),(x4_p,y4_p)),(excel_col(int(x2))+str(int(y2)),excel_col(x4_p)+str(y4_p)), calc_cost(d2), new_coords_pos]
                move_df = recursive_swing((x2,y2),g_pos_stack,"pos",gr,move_df,coords,(x2,y2))

            if g_neg_stack != (0,0,0,0,0) and inbounds((x2,y2), (g_neg_stack[0],g_neg_stack[1]), d2, calc_slope(x2,y2,g_neg_stack[0],g_neg_stack[1])):
                g_n = g_neg_stack[4]
                for l in lps:
                    x4_n = l[0]
                    y4_n = l[1]
                    g2_n = calc_angle((a,b),(x2,y2),(x4_n,y4_n))
                    if 0 >= g2_n >= g_n and (x4_n,y4_n) != (a,b):
                        new_coords_neg = list(coords)
                        new_coords_neg[:] = [x for x in coords if x != (x2,y2)]
                        new_coords_neg.append((x4_n,y4_n))
                        move_df.loc[len(move_df)] = [((x2,y2),(x4_n,y4_n)),(excel_col(int(x2))+str(int(y2)),excel_col(x4_n)+str(y4_n)), calc_cost(d2), new_coords_neg]
                move_df = recursive_swing((x2,y2),g_neg_stack,"neg",gr,move_df,coords,(x2,y2))
    return move_df

In [165]:
move_df = pd.DataFrame(columns=["move_coords","move_notational","cost","graph_coords"])
coords = [(1.0, 1.0), (3.0, 20.0), (5.0, 20.0), (11.0, 20.0), (15.0, 20.0), (17.0, 20.0), (18.0, 19.0), (19.0, 19.0), (8.0, 18.0), (14.0, 18.0), (16.0, 18.0), (19.0, 18.0), (1.0, 17.0), (9.0, 17.0), (15.0, 17.0), (20.0, 17.0), (4.0, 16.0), (5.0, 16.0), (12.0, 16.0), (18.0, 16.0), (16.0, 15.0), (20.0, 15.0), (13.0, 14.0), (8.0, 13.0), (16.0, 13.0), (19.0, 13.0), (20.0, 12.0), (1.0, 11.0), (4.0, 11.0), (16.0, 9.0), (5.0, 8.0), (8.0, 8.0), (12.0, 7.0), (18.0, 7.0), (16.0, 6.0), (14.0, 5.0), (20.0, 5.0), (12.0, 4.0), (17.0, 4.0), (10.0, 3.0), (8.0, 2.0), (15.0, 2.0), (1.0, 20.0), (6.0, 1.0), (20.0, 1.0)]

directory = "/Users/jnielsen/Desktop/May18_Puzzle/1_1/"
if not os.path.exists(directory):
    os.makedirs(directory)

master_df = find_moves(1,1,coords, move_df)
master_df.to_csv(directory+str(1)+"_"+str(1)+".csv")


def output_results(directory):

    for filename in os.listdir(directory):
        if filename.endswith(".csv"): 

            csv = os.path.join(directory, filename)
            csv_df = pd.read_csv(csv)
            
            new_dir_list = []

            for i in csv_df.iterrows():

                move_df = pd.DataFrame(columns=["move_coords","move_notational","cost","graph_coords"])

                index = i[0]
                row = i[1]
                move_coords = literal_eval(row[1])
                a = move_coords[1][0]
                b = move_coords[1][1]
                move_notational = literal_eval(row[2])
                cost = float(row[3])
                graph_coords = literal_eval(row[4])
                
#             print(move_coords)
#             print(a)
#             print(b)
#             print(move_notational)
#             print(cost)
#             print(graph_coords)
#             print("_____")

                if len(graph_coords) > 0:
            
                    store_df = find_moves(a,b,graph_coords, move_df)
                    new_dir = directory+str(a)+"_"+str(b)+"/"
                    new_dir_list.append(new_dir)
                    
                    if not os.path.exists(new_dir):
                        os.makedirs(new_dir)
                    
                    if len(store_df) > 0:
                        if str(a)+"_"+str(b) not in directory:
                            if move_notational[1] == "T20":
                                store_df.to_csv(new_dir+str(a)+"_"+str(b)+"_WINNER.csv")
                            else:
                                store_df.to_csv(new_dir+str(a)+"_"+str(b)+".csv")

            for i in new_dir_list:
                output_results(i)

output_results(directory)

KeyboardInterrupt: 